# GA Data Science 18 (DAT18) - Lab4

## Lab goals

- Simple APIs with Python
- Kimono Labs: an external tool for aquiring data

In [ ]:
# Imports at the top
import json
import urllib
import pandas as pd

import requests
import json


## Using Python APIs

### Either Use Sheetsu or an API you've found
Sheetsu: Turning Google Docs into APIs/
https://sheetsu.com/

### Using Sheetsu
I've created a GoogleDoc with wine data

https://docs.google.com/spreadsheets/d/1_U-DHIrBXN8A1e3NbyOIJRORge2udmmaa8qS9mKddWo/edit#gid=0 
https://sheetsu.com/apis/79656330

In [ ]:
# You can either post or get info from this API
# using the /column/Name you can pull only specfic subsets of the data
api_base_url = 'https://sheetsu.com/apis/79656330'
api_column = '/column/Name'


##### Reading Data from an API datafeed

In [ ]:
# What kind of data is this returning?
api_response = requests.get(api_base_url)
api_response.text

In [ ]:
# What kind of data is this returning?
api_column_url = api_base_url + api_column
column_response = requests.get(api_column_url)
column_response.text

#### Reading JSON

In [ ]:
reponse_dict = json.loads(api_response.text)

#### What is the response of an API and what does it mean??

In [ ]:
# What data was returned?
reponse_dict

#### Lets read the data into a DataFrame!

In [ ]:
wine_df = pd.DataFrame(reponse_dict['result'])
wine_df.head()

#### Pandas has great functions. We could just do it this way

This sometimes works, but the data may need adjusting

In [ ]:
pd.read_json(api_response.text).head(2)

In [ ]:
wine_df = pd.DataFrame(list(pd.read_json(api_response.text).result.values))
wine_df.head(2)

#### Writing to an API using PUT
You will rarely need to do this

In [ ]:
# Send data to an API
post_data = {
    'Grape' : ''
    , 'Name' : 'Dylans Wine'
    , 'Color' : 'R'
    , 'Country' : 'US'
    , 'Region' : 'CA'
    , 'Vinyard' : ''
    , 'Score' : '10'
    , 'Consumed In' : '2015'
    , 'Vintage' : '1973'
    , 'Price' : '200'
    }
requests.post(api_base_url, data=post_data)


### Exercise 1
    - Either with your own google doc or a different API
    - Extract some data using requests
    - Pull the data into a DataFrame
    - Search For missing data
        - Is there any missing data you can clean?
        - Is there any data you can just remove?
    - Summarize the data 
        - Try using describe, min, max, mean, var
        - What are the unique values and can you do anything with the text?
            Hint: Try using pd.get_dummies
    - Bonus: Create some visualizations form this data
    - Bonus: Try using sheetsu to push new data to the google doc (requests.post)

## Kimono Labs



Web scraping with the Kimono Labs API

https://www.kimonolabs.com/

Kimono labs has undergone a bit of a redesign in recent months and now encourages crawling for all the data you'd want ahead of time and extracting the batch in a series of calls. To create an api, follow the kimono labs walkthrough. To generate a large amount of data like we will here, a series of screenshot walkthroughs have been added.

After creating an API from an advanced search query as walked through in class, navigate to the Crawl Setup tab from the API management page in kimono.

In [ ]:
from IPython.display import Image
Image(filename='KL_Crawl_Setup.png') 

Change the crawl strategy to Generated URL List

In [ ]:
from IPython.display import Image
Image(filename='KL_Generated_URL_List.png') 

From here you can automatically generate a list of pages to extract data from, with the caveat that the pages you are navigating all have roughly the same structure. Here we are going to look for the top boxoffice_gross_us from 1990 to 2014. 

In [ ]:
from IPython.display import Image
Image(filename='KL_Parameter_range.png') 

In [ ]:
#After you let the data crawl the page, you can update the data here.

import json
import urllib

results = json.load(urllib.urlopen("https://www.kimonolabs.com/api/eb81bu78?apikey=4WrTjTsFDGmHCLnYrAXeSLIjJYcfrG7j"))
results

In [ ]:
your_key = "4WrTjTsFDGmHCLnYrAXeSLIjJYcfrG7j"
your_endpoint = "eb81bu78"
import json
import urllib
import pandas as pd

def getMovies(api_key=your_key,endpoint=your_endpoint):
    """
    Creates list of top 50 movies by gross box office
    sales for a with ratings and sales
    """
    
    movies, ratings, sales, years = [], [], [], []
    #Remember to replace this link with the link to your specifc API
    url = "https://www.kimonolabs.com/api/{}?".format(endpoint) + \
            "apikey={}".format(api_key)
    data = json.load(urllib.urlopen(url))
    
    # Iterate through json object to collect data
    for n in xrange(data['count']):
        n_title = data['results']['collection1'][n]['title']['text']
        n_rating = float(data['results']['collection1'][n]['rating'])
        n_sales = float(data['results']['collection1'][n]['sales'][1:-1])
        n_year = int(data['results']['collection1'][n]['url'][-4:])
        movies.append(n_title)
        ratings.append(n_rating)
        sales.append(n_sales)
        years.append(n_year)
    data = pd.DataFrame({'movie':movies,'rating':ratings,'sales_in_M':sales,'year':years})
    
    return data
movies = getMovies()
movies.head()

Note a few things: The function we wrote relies on extracting pieces of a string and converting them to numeric types. Here, the year can be extracted from the url used to generate the movies, so we use the slice `[-4:]` to identify that. Similarly, the gross cost comes out as $##M, so we know we never want the first or last value. Thus the slice `[1:-1]` is correct.

###Exercise 2.1
Build your own api via kimono at https://www.kimonolabs.com/. Create the api we created in class to call the data.

What were the average sales in 1995?
What was the average rating?

###Bonus
What were the average sales for the 90s? How does that differ from the 2000s?